# Future Energy Outlook - Models

This notebook introduces FEO's `Model` class. A FEO _Model_ (short for 'Systems Model') sets the scope and boundaries of an workflow using the FEO platform. A FEO model is scoped with the following properties:
- **sectors**: a list of sectors covered by the FEO platform, e.g. 'power', or 'land'.
- **nodes**: a list of nodes, corresponding to a geographical unit (like a country or state/province), a grid-region, or even a physical asset.
- **links**: a list of links between nodes, which allow the transmission or flow of a given commodity. These links might represent, as an example, interconnectors between grid regions.
- **time_scope**: the time definition of the model. Sometimes, instead of contiguous time, models are defined with representative time-slices, trading temporal-fidelity for longer time horizons or model complexity.

FEO Models also have some identifying nameplate data, including:
- **uuid**: a unique unit ID for the model used by the FEO platform.
- **name**: a descriptive name for the model.
- **slug**: a url-parseable representation of the model name, useful for link sharing and human-readability 
- **version**: the version hash of the model, allowing full reproduceability.

Lastly, users can share their FEO Models. FEO Models have the following properties for access control and sharing:
- **owner_id**: The user-id associated with the model
- **public**: Whether or not a FEO model can be be publicly accessed.

This notebook shows how to use the FEO client to create, update, and delete `Model` objects.

### Authenticating the FEO Client

The FEO client is packaged with a commandline utility for special operations with the FEO platform like logging in. To log in to the FEO platform from your device, use the command-line login utility:

    feo auth login
    
More information can be found in the [quickstart guide.](link)

### Using the FEO client

First, load the object-oriented `Model` client.

In [ ]:
from feo.client import Model

The `Model` client can be used to search the FEO platform and instantiate copies of itself.

#### Searching for Models

Models can be searched using the `.search` method. This method has the following parameters:
    
    public: bool
        Models can be filtered according to wethey they are public.
    owner: str
        Models can be searched for from a specific author.
    page: int
        Set the page offset for the search results
    limit: int
        Set the number of search results to be returned (max: XX)

In [ ]:
search_results = Model.search(public=True)

In [ ]:
for model in search_results:
    print (model.uuid, model.name)

#### Creating a new model

A new model can be constructed in-memory simply by calling the client object.

In [ ]:
my_model = Model(
    name="My new Model",
    sectors=["power"],
    time_scope={
        "years":[2025,2030],
        "year-parts":6,
        "day-parts":24,
    },
    version="0.01",
    node_ids=["IND-NE","IND-EA","IND-NO"],
    link_ids=["IND-NE:IND-EA","IND-NE:IND-NO","IND-EA:IND-NO"],
)

For simplicity, instead of `node_ids` and `link_ids`, a model can be specified by its `geography` and `node_level`. For example, to obtain a model of the 16 federal states of Germany, i.e. resolved at the 'admin-1' (i.e. state/province) level, the geography of `germany` can be supplied, with the node_level of `admin_1`.

The Model client will find the node corresponding to the `geography` alias, and will specify the model with the children of this node at the specified `node_level`.

In [ ]:
my_deu_model = Model(
    name="My German Model",
    sectors=["power"],
    time_scope={
        "years":[2025,2030],
        "year-parts":6,
        "day-parts":24,
    },
    version="0.01",
    geography="germany",
    node_level="admin_1",
)

#### Saving your model to the FEO platform

If you've just created your model, it is not yet synced with the FEO platform. To save your model, use the `.save` method. You model will then have a `uuid` and `slug` provided by the FEO platform.

In [ ]:
print (my_deu_model.uuid, my_deu_model.slug)
my_deu_model.save()
print (my_deu_model.uuid, my_deu_model.slug)

#### Editing your model
If your model doesn't have any dependent object yet (i.e. any `scenarios` or `runs`), it can still be editted. Simply change your desired parameter and then `.save` again.

In [ ]:
my_deu_model.time_scope['years'].append(2035)
my_deu_model.save()
print (my_deu_model.time_scope['years'])

#### Copying a model

You may find a public model by another author that you want to build on top of. You cannot build directly on top of another user's model, but you can copy their model to your own version. Use the `.copy` method to acheive this.
Don't forget to save your model!

In [ ]:
my_copied_model = search_results[0].copy()
my_copied_model.save()